In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, f1_score, precision_score, recall_score, log_loss

In [3]:
from explainerdashboard.explainers import *
from explainerdashboard.dashboards import *
from explainerdashboard.datasets import *

# RandomForestClassifier dashboard

In [34]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestClassifierBunch(model, X_test, y_test, metric=roc_auc_score, 
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, #names of passengers 
                               labels=['Not survived', 'Survived'])

In [35]:
explainer.plot_importances(cats=True)

Calculating shap values...
Generating shap TreeExplainer...


In [37]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        decision_trees=True)
db.run(8052)

Calculating predictions...
Calculating pred_percentiles...
Calculating prediction probabilities...
Calculating importances...
Calculating shap interaction values...
Generating ShadowDecTree for each individual decision tree...
Running Model Explainer on http://localhost:8052
 * Serving Flask app "explainerdashboard.dashboards" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Feb/2020 15:09:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 15:09:31] "POST /_dash-update-component

# RandomForestRegressor example

In [ ]:
X_train, y_train, X_test, y_test = titanic_fare()
train_names, test_names = titanic_names()

model = RandomForestRegressor(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

explainer = RandomForestRegressionBunch(model, X_test, y_test, r2_score,
                               cats=['Sex', 'Deck', 'Embarked'],
                               idxs=test_names, units="$")

In [ ]:
db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=True,
                        shadow_trees=True)
db.run(8052)

# XGBClassifier example

In [ ]:
X_train, y_train, X_test, y_test = titanic_survive()
train_names, test_names = titanic_names()

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

explainer = TreeClassifierBunch(xgb_model, X_test, y_test, roc_auc_score,
                                   cats=['Sex', 'Cabin', 'Embarked'],
                                   idxs=test_names, #names of passengers 
                                   labels=['Not survived', 'Survived'])

db = ExplainerDashboard(explainer,
                        model_summary=True,
                        contributions=True,
                        shap_dependence=True,
                        shap_interaction=False)
db.run(8052)